**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery).**

---


# Introduction

The first test of your new data exploration skills uses data describing crime in the city of Chicago.

Before you get started, run the following cell. It sets up the automated feedback system to review your answers.

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex1 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Use the next code cell to fetch the dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Count tables in the dataset

How many tables are in the Chicago Crime dataset?

In [3]:
# Write the code you need here to figure out the answer

# List all the tables in the "chicago_crime" dataset
tables = list(client.list_tables(dataset))

# Print number of tables in the dataset
print(len(tables))

1


In [4]:
num_tables = len(tables)  # Store the answer as num_tables and then run this cell

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [5]:
#q_1.hint()
#q_1.solution()

### 2) Explore the table schema

How many columns in the `crime` table have `TIMESTAMP` data?

In [6]:
# Write the code to figure out the answer

# Construct a reference to the "crime" table
table_ref = dataset_ref.table("crime")

# API request - fetch the table
table = client.get_table(table_ref)

# Print information on all the columns in the "crime" table in the "chicago_crime" dataset
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', 'Unique identifier for the record.', (), None),
 SchemaField('case_number', 'STRING', 'NULLABLE', 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.', (), None),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', 'Date when the incident occurred. this is sometimes a best estimate.', (), None),
 SchemaField('block', 'STRING', 'NULLABLE', 'The partially redacted address where the incident occurred, placing it on the same block as the actual address.', (), None),
 SchemaField('iucr', 'STRING', 'NULLABLE', 'The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.', (), None),
 SchemaField('primary_type', 'STRING', 'NULLABLE', 'The primary description of the IUCR code.', (), None),
 SchemaField('description', 'STRING', 'NULLABLE', 'The secondary description of the IUCR code, a s

In [7]:
# Print count columns in the crime table have TIMESTAMP data
print(sum(table.field_type == "TIMESTAMP" for table in table.schema))

2


In [8]:
num_timestamp_fields = sum(table.field_type == "TIMESTAMP" for table in table.schema) # Put your answer here

# Check your answer
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [9]:
#q_2.hint()
#q_2.solution()

### 3) Create a crime map

If you wanted to create a map with a dot at the location of each crime, what are the names of the two fields you likely need to pull out of the `crime` table to plot the crimes on a map?

In [10]:
# Write the code here to explore the data so you can find the answer

# Preview the first five lines of the "crime" table
client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  after removing the cwd from sys.path.


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,10227340,HY414853,2015-09-08 00:05:00+00:00,059XX W HIGGINS AVE,0930,MOTOR VEHICLE THEFT,THEFT/RECOVERY: AUTOMOBILE,STREET,False,False,...,45,11,07,1135391.0,1933234.0,2015,2018-02-10 15:50:01+00:00,41.972980,-87.777521,"(41.972979743, -87.777521099)"
1,10233446,HY421617,2015-09-12 23:00:00+00:00,059XX S NASHVILLE AVE,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,ALLEY,False,False,...,23,64,14,1133415.0,1864487.0,2015,2018-02-10 15:50:01+00:00,41.784363,-87.786398,"(41.784363241, -87.786398395)"
2,10235028,HY422845,2015-09-14 12:00:00+00:00,046XX W FOSTER AVE,0330,ROBBERY,AGGRAVATED,FOREST PRESERVE,False,False,...,39,13,03,1144609.0,1934168.0,2015,2018-02-10 15:50:01+00:00,41.975374,-87.743600,"(41.975373594, -87.743600482)"
3,10235256,HY423025,2015-09-13 17:00:00+00:00,055XX N OVERHILL AVE,1345,CRIMINAL DAMAGE,TO CITY OF CHICAGO PROPERTY,STREET,False,False,...,41,10,14,1124004.0,1936017.0,2015,2018-02-10 15:50:01+00:00,41.980812,-87.819333,"(41.980811688, -87.819333028)"
4,10239920,HY427653,2015-09-17 21:33:00+00:00,020XX E 87TH ST,1812,NARCOTICS,POSS: CANNABIS MORE THAN 30GMS,STREET,True,False,...,8,45,18,1191437.0,1847696.0,2015,2018-02-10 15:50:01+00:00,41.737078,-87.574211,"(41.737077626, -87.574211133)"


In [11]:
fields_for_plotting = ['latitude', 'longitude'] # Put your answers here

# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [12]:
#q_3.hint()
#q_3.solution()

Thinking about the question above, there are a few columns that appear to have geographic data. Look at a few values (with the `list_rows()` command) to see if you can determine their relationship.  Two columns will still be hard to interpret. But it should be obvious how the `location` column relates to `latitude` and `longitude`.

In [13]:
# Scratch space for your code

# Preview the first ten lines of the "crime" table
client.list_rows(table, max_results=10).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  after removing the cwd from sys.path.


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,10227340,HY414853,2015-09-08 00:05:00+00:00,059XX W HIGGINS AVE,0930,MOTOR VEHICLE THEFT,THEFT/RECOVERY: AUTOMOBILE,STREET,False,False,...,45,11,07,1135391.0,1933234.0,2015,2018-02-10 15:50:01+00:00,41.972980,-87.777521,"(41.972979743, -87.777521099)"
1,10233446,HY421617,2015-09-12 23:00:00+00:00,059XX S NASHVILLE AVE,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,ALLEY,False,False,...,23,64,14,1133415.0,1864487.0,2015,2018-02-10 15:50:01+00:00,41.784363,-87.786398,"(41.784363241, -87.786398395)"
2,10235028,HY422845,2015-09-14 12:00:00+00:00,046XX W FOSTER AVE,0330,ROBBERY,AGGRAVATED,FOREST PRESERVE,False,False,...,39,13,03,1144609.0,1934168.0,2015,2018-02-10 15:50:01+00:00,41.975374,-87.743600,"(41.975373594, -87.743600482)"
3,10235256,HY423025,2015-09-13 17:00:00+00:00,055XX N OVERHILL AVE,1345,CRIMINAL DAMAGE,TO CITY OF CHICAGO PROPERTY,STREET,False,False,...,41,10,14,1124004.0,1936017.0,2015,2018-02-10 15:50:01+00:00,41.980812,-87.819333,"(41.980811688, -87.819333028)"
4,10239920,HY427653,2015-09-17 21:33:00+00:00,020XX E 87TH ST,1812,NARCOTICS,POSS: CANNABIS MORE THAN 30GMS,STREET,True,False,...,8,45,18,1191437.0,1847696.0,2015,2018-02-10 15:50:01+00:00,41.737078,-87.574211,"(41.737077626, -87.574211133)"
5,10248814,HY436261,2015-09-24 10:15:00+00:00,056XX W 64TH PL,3970,INTIMIDATION,EXTORTION,RESIDENCE,False,False,...,13,64,26,1139892.0,1861229.0,2015,2018-02-10 15:50:01+00:00,41.775307,-87.762730,"(41.775307198, -87.762729918)"
6,10276301,HY464013,2015-10-14 09:00:00+00:00,036XX W 56TH PL,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,ABANDONED BUILDING,False,False,...,14,62,14,1153115.0,1866902.0,2015,2018-02-10 15:50:01+00:00,41.790624,-87.714106,"(41.790623788, -87.714105569)"
7,10000153,HY189345,2015-03-18 12:20:00+00:00,091XX S UNIVERSITY AVE,0483,BATTERY,AGG PRO.EMP: OTHER DANG WEAPON,"SCHOOL, PUBLIC, BUILDING",True,False,...,8,47,04B,1185475.0,1844606.0,2015,2018-02-10 15:50:01+00:00,41.728741,-87.596151,"(41.728740563, -87.596150779)"
8,10328388,HY519212,2015-11-29 23:00:00+00:00,040XX W 59TH ST,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,RESIDENCE,False,False,...,13,62,11,1150260.0,1865168.0,2015,2018-02-10 15:50:01+00:00,41.785921,-87.724619,"(41.785921376, -87.724619292)"
9,10199133,HY387031,2015-07-25 00:01:00+00:00,054XX N MENARD AVE,1750,OFFENSE INVOLVING CHILDREN,CHILD ABUSE,RESIDENCE,True,False,...,45,11,20,1136536.0,1935452.0,2015,2018-02-10 15:50:01+00:00,41.979046,-87.773257,"(41.979045669, -87.773257296)"


# Keep going

You've looked at the schema, but you haven't yet done anything exciting with the data itself. Things get more interesting when you get to the data, so keep going to **[write your first SQL query](https://www.kaggle.com/dansbecker/select-from-where).**

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*